# Filtering Deep Dive

In this tutorial, we will explore the power of nornir filtering and demonstrate why nornir is a first-class inventory management framework.

This tutorial will demonstrate some common network use-cases and how nornir filtering can be target to target hosts or groups with precision.

Firstly, let's start with initialising our nornir inventory.

## Tutorial Inventory

Firstly, let's start with initialising our nornir inventory.

In [ ]:
# Import modules
from nornir import InitNornir
from nornir.core.filter import F

# Initialise nornir
nr = InitNornir(config_file="filtering_deep_dive/config.yaml")

Next, let's look at our tutorial `hosts.yaml` file. Don't be overwhelmed by the size of the inventory or the content within as we will explore this in further detail later in the tutorial:

In [2]:
%cat filtering_deep_dive/inventory/hosts.yaml

---
lab-csr-011.lab.norn.local:
    hostname: lab-csr-011.lab.norn.local
    groups:
        - ios
        - lab
        - mel
    data:
        mgmt_ip: 10.0.0.16
        vendor: cisco
        device_type: router
        os_version: 16.6.4
        site_code: mel
        
dfjt-r001.lab.norn.local:
    hostname: dfjt-r001.lab.norn.local
    groups:
        - ios
        - lab
        - bcn
    data:
        mgmt_ip: 10.0.0.1
        vendor: cisco
        device_type: router
        os_version: 16.6.3
        site_code: bcn
        
lab-arista-01.lab.norn.local:
    hostname: lab-arista-01.lab.norn.local
    groups:
        - eos
        - lab
        - mtl
    data:
        mgmt_ip: 10.0.0.11
        vendor: arista
        device_type: switch
        os_version: 4.22.0F
        site_code: mtl
        
lab-arista-02.lab.norn.local:
    hostname: lab-arista-02.lab.norn.local
    groups:
        - eos
        - lab
        - mel
    data:
        mgmt_ip: 10.0.0.18
        vendor: arista
 

Next, let's look at our tutorial `groups.yaml` file. There are a mixture of group types in here.

Some appear to be based on network operating systems, some on operating environments and others based on physical location. nornir has no pre-conceived ideas or limits on group composition or structures:

In [1]:
%cat filtering_deep_dive/inventory/groups.yaml

---
ios:
    platform: ios
    data:
        vendor: cisco
junos:
    platform: junos
    data:
        vendor: juniper
eos:
    platform: eos
    data:
        vendor: arista
nxos:
    platform: nxos
    data:
        vendor: cisco
nxos_ssh:
    platform: nxos_ssh
    data:
        vendor: cisco
panos:
    platform: paloalto_panos
    data:
        vendor: palo alto
lab:
    data:
        sla: 70
        production: false
prod:
    data:
        sla: 90
        production: true
test:
    data:
        sla: 80
        production: false
mel:
    data:
        full_name: Melbourne
        country: Australia
        region: apac
        hemisphere: southern
        site_type: primary
hbt:
    data:
        full_name: Hobart
        country: Australia
        region: apac
        hemisphere: southern
        site_type: tertiary
chc:
    data:
        full_name: Christchurch
        country: New Zealand
        region: apac
        hemisphere: southern
        site_type: secondary
ptl:
    

## Custom inventory data

Before we dive into filtering, it's important to introduce some core concepts regarding custom data.

nornir allows you to populate custom data on `hosts` or `groups` objects under the `data` key, in whatever key/value data structure you choose. You can name these keys whatever you like to suit your business needs.

Firstly, let's explore an extract of one host and one group in the initial inventory introduced `hosts.yaml` and `groups.yaml` file. 

**NOTE: These files have been cut down to only contain one entry each, and also have some comments dispersed for readability**

In [4]:
%cat filtering_deep_dive/inventory/hosts_extract.yaml

---
lab-csr-011.lab.norn.local:
    hostname: lab-csr-011.lab.norn.local
    groups:
        - ios
        - lab
        - mel
    data: # Anything under this key is custom data
        mgmt_ip: 10.0.0.16 # This is custom data
        vendor: cisco # So is this
        device_type: router # Same as this
        os_version: 16.6.4 # Also this too
        site_code: mel # Yes, and also this

In [5]:
%cat filtering_deep_dive/inventory/groups_extract.yaml

---
mel:
    data: # Anything under this key is custom data
        full_name: Melbourne # This is custom data
        country: Australia # So is this
        region: apac # Same as this
        hemisphere: southern # Also this too
        site_type: primary # Yes, and also this


As you can see in the two examples above, you can use custom data in any manner you like to record any information you please. The examples above are a "flat" data structure under the `data` key, but you can nest your data structure in any key/value structure to suit your needs. 

In the below example, we take the `mgmt_ip` value of `10.0.0.16`, and reorient it under an alternate data structure to the key `mgmt` which could allow for future expansion:

In [6]:
%cat filtering_deep_dive/inventory/hosts_extract_alternate.yaml

---
lab-csr-011.lab.norn.local:
    hostname: lab-csr-011.lab.norn.local
    groups:
        - ios
        - lab
        - mel
    data: # Anything under this key is custom data
        ip_addresses:
            mgmt: 10.0.0.16 # Alternate way of managing mgmt_ip data
        vendor: cisco 
        device_type: router 
        os_version: 16.6.4 
        site_code: mel

Naturally over time, you needs to store custom data might change and ideally you enrich your custom data with as much business information as possible. 

Below shows an example where a new set of key/value pairs related to the location of the site are now stored under the `location` key. This means that any existing code leveraging the existing data structure doesn't need to be refactored:

In [7]:
%cat filtering_deep_dive/inventory/groups_extract_alternate.yaml

---
mel:
    data: # Anything under this key is custom data
        full_name: Melbourne 
        country: Australia 
        region: apac 
        hemisphere: southern
        site_type: primary
        location: # New location data is stored about the site
            address: 1 Wurundjeri Street
            suburb: Northcote
            zip_code: 3070




### Viewing host/group data

Before we revert back to filtering, we will show you how to access all data which is accessible or attributed to a `host` or `group`.

First, we will initialise nornir and filter the inventory on a single host `
lab-csr-011.lab.norn.local`: 

In [35]:
# Import modules
from nornir import InitNornir
from nornir.core.filter import F

# Initialise nornir
nr = InitNornir(config_file="filtering_deep_dive/config.yaml")
# Print the number of hosts in the inventory
print(f"Number of hosts in entire inventory: {len(nr.inventory.hosts)}")
# Filter the entire host inventory on one host
single_host = nr.inventory.hosts['lab-csr-011.lab.norn.local']
# Print out the host
print(f"Single host: {single_host}")

Number of hosts in entire inventory: 27
Single host: lab-csr-011.lab.norn.local


We can now print out all the data associated to the host `lab-csr-011.lab.norn.local` by dumping the dictionary structure:

In [36]:
# Import the JSON module
import json
# Dump the dictionary, assign to variable
single_host_data = json.dumps(single_host.dict(), indent=2)
# Print seperator
print("=" * 50)
# Print header and host data structure
print(f"Displaying information for host: {single_host}")
print(f"{single_host_data}")
# Print seperator
print("=" * 50)

Displaying information for host: lab-csr-011.lab.norn.local
{
  "name": "lab-csr-011.lab.norn.local",
  "connection_options": {},
  "groups": [
    "ios",
    "lab",
    "mel"
  ],
  "data": {
    "mgmt_ip": "10.0.0.16",
    "vendor": "cisco",
    "device_type": "router",
    "os_version": "16.6.4",
    "site_code": "mel"
  },
  "hostname": "lab-csr-011.lab.norn.local",
  "port": null,
  "username": null,
  "password": null,
  "platform": null
}


As you can see above, all our data related to the host is available for viewing and usage in any subsequent code. Whether we are using JSON or YAML, the data structure remains the same.

Below are some examples for displaying the values of some of the keys:

In [30]:
print(f"Displaying data for host: {single_host}")
# Access the site_code value, nested under the data key
single_host_site_code = single_host.dict()["data"]["site_code"]
print(f"Site code is: {single_host_site_code}")
# Access the platform value
single_host_platform = single_host.dict()["platform"]
print(f"Platform is: {single_host_platform}")
# Access the os_version value, nested under the data key
single_host_os_version = single_host.dict()["data"]["os_version"]
print(f"OS Version is: {single_host_os_version}")

Displaying data for host: lab-csr-011.lab.norn.local
Site code is: mel
Platform is: None
OS Version is: 16.6.4


The same concept shown above with a `host`, also applies to a `group`. 

As we have described how this works, we will show the same concepts as they apply to groups below, with comments:

In [34]:
# Import modules
from nornir import InitNornir
from nornir.core.filter import F
import json

# Initialise nornir
nr = InitNornir(config_file="filtering_deep_dive/config.yaml")
# Print the number of groups in the inventory
print(f"Number of groups in entire inventory: {len(nr.inventory.groups)}")
# Filter the entire group inventory on one group
single_group = nr.inventory.groups['ptl']
# Print out the group
print(f"Single group: {single_group}")

# Dump the dictionary, assign to variable
single_group_data = json.dumps(single_group.dict(), indent=2)
# Print seperator
print("=" * 50)
# Print header and host data structure
print(f"Displaying information for group: {single_group}")
print(f"{single_group_data}")
# Print seperator
print("=" * 50)

print(f"Displaying data for group: {single_group}")
# Access the full_name value, nested under the data key
single_group_site_code = single_group.dict()["data"]["full_name"]
print(f"Full Name is: {single_group_site_code}")
# Access the platform value
single_group_platform = single_group.dict()["platform"]
print(f"Platform is: {single_group_platform}")
# Access the os_version value, nested under the data key
single_group_region = single_group.dict()["data"]["region"]
print(f"Region is: {single_group_region}")

Number of groups in entire inventory: 15
Single group: ptl
Displaying information for group: ptl
{
  "name": "ptl",
  "connection_options": {},
  "groups": [],
  "data": {
    "full_name": "Port Louis",
    "country": "Mauritius",
    "region": "amea",
    "hemisphere": "southern",
    "site_type": "primary"
  },
  "hostname": null,
  "port": null,
  "username": null,
  "password": null,
  "platform": null
}
Displaying data for group: ptl
Full Name is: Port Louis
Platform is: None
Region is: amea


By now, you should have an understanding of the following regarding inventory custom data:
    - What is nornir custom data?  
    - How can it be stored?  
    - How can I access it?  
    - How can I view and troubleshoot it?  
    
In the next section, we will now revert back to nornir filtering, which will build off your understanding of these concepts.